In [3]:
# #importing libraries
# import warnings
# warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing import text
from tensorflow.keras.utils import to_categorical as np_utils
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [4]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [5]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [6]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [7]:
#model building
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
cbow.fit(x, y, epochs=10, verbose=1)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 4.3417 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 4.3260 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 4.3103 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 4.2946 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 4.2790 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 4.2633 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 4.2477 - accuracy: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 4.2321 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 11ms/step - loss: 4.2165 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 4.2008 - accuracy: 1.0000


In [9]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.024132,-0.042009,0.024559,-0.013954,-0.029109,0.021673,0.004279,-0.022791,0.020432,0.018128,...,-0.007464,0.047869,0.039061,-0.019290,-0.015061,0.009768,0.041710,0.026451,-0.005080,-0.014420
networks,-0.046398,-0.033725,-0.016914,0.048352,0.029887,0.029180,-0.044961,-0.037505,0.048768,-0.044508,...,0.046418,0.030682,0.041191,-0.045340,0.006757,0.008462,0.021942,0.039475,-0.045283,0.037872
neural,0.027256,-0.019561,0.044410,0.039478,-0.015387,-0.017385,-0.018509,-0.049695,-0.025414,0.041414,...,-0.041284,-0.004338,-0.012553,-0.000168,-0.049402,0.020657,0.027419,0.046027,0.040403,-0.003116
and,0.022697,0.017778,0.017083,0.045327,-0.028970,0.001203,0.021739,-0.049963,-0.023419,0.018857,...,-0.023548,0.019224,-0.012165,0.048842,-0.017029,0.017034,0.016299,-0.021936,0.022835,-0.020572
as,-0.039427,0.046484,0.013791,0.022227,-0.025365,-0.020873,-0.002624,0.037672,0.039645,0.000126,...,0.003424,-0.047524,0.006624,-0.034221,0.019509,0.016030,-0.014215,-0.020435,0.008452,-0.014945


In [11]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)


similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['networks']}

similar_words

(74, 74)


{'networks': ['in', 'learning', 'game', 'applied', 'medical']}